<a href="https://colab.research.google.com/github/hfarazul/Base_developer_copilot/blob/main/Document_creator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [7]:
from crewai_tools import tool
import requests
from bs4 import BeautifulSoup

@tool
def ScrapeWebsiteTool(website_url: str) -> str:
    """
    Scrapes the content of a given website.

    Args:
        website_url: The URL of the website to scrape.

    Returns:
        The all the text content  of the website.
    """
    response = requests.get(website_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    content = soup.get_text()
    return content

In [25]:
from crewai_tools import tool
import requests
from bs4 import BeautifulSoup

@tool
def ScrapeWebsiteTool(website_url: str) -> str:
    """
    Scrapes the content of a given website.

    Args:
        website_url: The URL of the website to scrape.

    Returns:
        The all the text content  of the website.
    """
    response = requests.get(website_url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract headings, text, and code snippets
    content = ''

    # Extract headings
    for heading in soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6']):
        content += f"{heading.get_text()}\n"

    # Extract paragraphs
    for paragraph in soup.find_all('p'):
        content += f"{paragraph.get_text()}\n"

    # Extract code snippets
    for code in soup.find_all('code'):
        content += f"\n```\n{code.get_text()}\n```\n"

    return content

In [22]:
import os
os.environ["OPENAI_API_KEY"] = "APIKEY"
os.environ["SERPER_API_KEY"] = "APIKEY"


In [28]:

from crewai import Agent, Task, Crew, Process
from crewai_tools import ScrapeWebsiteTool, TXTSearchTool

# Set up API key

# Instantiate tools
scrape_website_tool = ScrapeWebsiteTool()
txt_search_tool = TXTSearchTool()

# Create agents
web_surfer = Agent(
    role='Web Surfer',
    goal='Extract all content, including text and code snippets, from all subpages of the given website',
    backstory='You are skilled at navigating websites and extracting comprehensive information.',
    tools=[scrape_website_tool],
    verbose=True
)

document_writer = Agent(
    role='Document Writer',
    goal='Compile extracted content into a structured document',
    backstory='You have a knack for organizing information in a clear and concise manner.',
    tools=[txt_search_tool],
    verbose=True
)

# Define tasks
web_scraping_task = Task(
    description='Visit each subpage of the website at {website_url} and extract all content, including text, headings, and code snippets.',
    expected_output='A collection of extracted content from all subpages.',
    tools=[scrape_website_tool],
    agent=web_surfer
)

document_compilation_task = Task(
    description='Organize the extracted content from {website_url} into a structured document.',
    expected_output='A well-organized document containing all the information from the website, including headings, text, and code snippets.',
    tools=[txt_search_tool],
    agent=document_writer,
    output_file='structured_document.md'  # The final document will be saved here
)

# Assemble a crew
crew = Crew(
    agents=[web_surfer, document_writer],
    tasks=[web_scraping_task, document_compilation_task],
    process=Process.sequential,
    verbose=2
)

# Execute tasks
crew.kickoff(inputs={'website_url': 'https://docs.crewai.com/'})


 [DEBUG]: == Working Agent: Web Surfer
 [INFO]: == Starting Task: Visit each subpage of the website at https://docs.crewai.com/ and extract all content, including text, headings, and code snippets.


> Entering new CrewAgentExecutor chain...
To begin, I should read the content of the main website to understand its layout and identify the subpages that I need to visit.

Action: 
Read website content

Action Input: 
{
  "website_url": "https://docs.crewai.com/"
}
 

crewAI
 Skip to content
 crewAI
 Home
 Initializing search
 crewAI
 crewAI
 crewAI
 Home
 Core Concepts
 Core Concepts
 Agents
 Tasks
 Tools
 Processes
 Crews
 Collaboration
 Memory
 Using LangChain Tools
 Using LlamaIndex Tools
 How to Guides
 How to Guides
 Installing CrewAI
 Getting Started
 Create Custom Tools
 Using Sequential Process
 Using Hierarchical Process
 Create your own Manager Agent
 Connecting to any LLM
 Customizing Agents
 Human Input on Execution
 Agent Monitoring with AgentOps
 Agent Monitoring with LangTr

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 8192 tokens. However, your messages resulted in 8210 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}